In [16]:
import numpy as np
from typing import Iterable

## explore data

In [17]:
!ls

delta_delta_p.xlsx     example_tree.txt  hw02_UpliftTreeRegressor_v01.ipynb
example_preds.npy      example_X.npy	 L2_models_for_lms.ipynb
example_treatment.npy  example_y.npy


### example of data to fit & preds

In [18]:
ex_X = np.load("example_X.npy")
ex_y = np.load("example_y.npy")
ex_treatment = np.load("example_treatment.npy")
ex_preds = np.load("example_X.npy")

In [19]:
print(ex_X.shape)
ex_X

(50000, 5)


array([[ 0.40072881,  0.8084128 ,  0.94338544,  0.19877038, -2.89635463],
       [ 1.11588933, -2.1070723 , -0.5123218 , -0.20680012,  0.70560314],
       [-0.15370419, -0.01068205, -0.38601986,  0.27470753, -0.27732682],
       ...,
       [-0.01556159,  2.64910009, -1.34283527,  1.08404747,  0.05824139],
       [-1.14064404,  0.65152198,  0.74822655, -1.59057866,  1.07153253],
       [ 1.60054183,  0.39258118, -0.26121784, -0.808533  , -0.3083781 ]])

In [20]:
print(ex_y.shape)
print(ex_y)
ex_y.size

(50000,)
[2.8480419  1.11098383 0.24938254 ... 3.88156137 0.25859638 0.50738829]


50000

In [21]:
print(ex_treatment.shape)
ex_treatment

(50000,)


array([1, 1, 1, ..., 1, 1, 0])

In [22]:
print(ex_preds.shape)
ex_preds

(50000, 5)


array([[ 0.40072881,  0.8084128 ,  0.94338544,  0.19877038, -2.89635463],
       [ 1.11588933, -2.1070723 , -0.5123218 , -0.20680012,  0.70560314],
       [-0.15370419, -0.01068205, -0.38601986,  0.27470753, -0.27732682],
       ...,
       [-0.01556159,  2.64910009, -1.34283527,  1.08404747,  0.05824139],
       [-1.14064404,  0.65152198,  0.74822655, -1.59057866,  1.07153253],
       [ 1.60054183,  0.39258118, -0.26121784, -0.808533  , -0.3083781 ]])

### description of tree

In [23]:
with open('example_tree.txt') as f:
    ex_tree_descr = f.read()
    print(ex_tree_descr)

Root
n_items: 50000
ATE: 0.8133621067075059
split_feat: feat0
split_threshold: 0.8428329389786856

	Left
	n_items: 40000
	ATE: 0.45661983599928
	split_feat: feat0
	split_threshold: -0.9878097589516122

		Left <leaf>
		n_items: 8000
		ATE: -0.708939125981635
		split_feat: None
		split_threshold: None

		Right
		n_items: 32000
		ATE: 0.7439039464385134
		split_feat: feat1
		split_threshold: 0.8401218986161383

			Left <leaf>
			n_items: 25600
			ATE: 0.5381690182796856
			split_feat: None
			split_threshold: None

			Right <leaf>
			n_items: 6400
			ATE: 1.5773329275902113
			split_feat: None
			split_threshold: None

	Right <leaf>
	n_items: 10000
	ATE: 2.2242964961219096
	split_feat: None
	split_threshold: None


## class draft

In [24]:
import numpy as np
from typing import Iterable

class UpliftTreeRegressor:
    
    ## init class instance
    def __init__(self,
        max_depth: int = 3, # максимальная глубина дерева.
        min_samples_leaf: int = 1000, # минимальное необходимое число обучающих объектов в листе дерева.
        min_samples_leaf_treated: int = 300, # минимальное необходимое число обучающих объектов с T=1 в листе дерева.
        min_samples_leaf_control: int = 300, # минимальное необходимое число обучающих объектов с T=0 в листе дерева.
    ):
        
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_leaf_treated = min_samples_leaf_treated
        self.min_samples_leaf_control = min_samples_leaf_control
        
    # fit the model
    def fit(
        self,
        X: np.ndarray, # массив (n * k) с признаками.
        treatment: np.ndarray, # массив (n) с флагом воздействия.
        y: np.ndarray # массив (n) с целевой переменной.
    ) -> None:
        
        pass
    
    def predict(self, X: np.ndarray) -> Iterable[float]:
        # compute predictions
        
        pass
#         return predictions
    
    
        

## class to send

In [32]:
import numpy as np
from typing import Iterable

class UpliftTreeRegressor:
    
    ## init class instance
    def __init__(self,
        max_depth: int = 3, # максимальная глубина дерева.
        min_samples_leaf: int = 1000, # минимальное необходимое число обучающих объектов в листе дерева.
        min_samples_leaf_treated: int = 300, # минимальное необходимое число обучающих объектов с T=1 в листе дерева.
        min_samples_leaf_control: int = 300, # минимальное необходимое число обучающих объектов с T=0 в листе дерева.
    ):
        
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_leaf_treated = min_samples_leaf_treated
        self.min_samples_leaf_control = min_samples_leaf_control
        
    # fit the model
    def fit(
        self,
        X: np.ndarray, # массив (n * k) с признаками.
        treatment: np.ndarray, # массив (n) с флагом воздействия.
        y: np.ndarray # массив (n) с целевой переменной.
    ) -> None:
        
        self.n_features_ = X.shape[1]
        self.tree = self._grow_tree(X, treatment, y)
    
    
    def _grow_tree(self, X, treatment, y, depth=0):
        
        predicted_ate = self._ATE(treatment, y)
        
        node = Node(
            n_items=y.size,
            ATE=predicted_ate,
        )
        
         # Split recursively until maximum depth is reached
        if depth < self.max_depth:
            idx, thr = self._best_split(X, treatment, y)
            
            if idx is not None:
                ## split data by threshold
                indices_left = X[:, idx] <= thr
                X_left, treatment_left, y_left = \
                    X[indices_left], treatment[indices_left], y[indices_left]
                X_right, treatment_right, y_right = \
                    X[~indices_left], treatment[~indices_left], y[~indices_left]
                
                node.split_feat = idx
                node.split_threshold = thr
                node.left = self._grow_tree(X_left, treatment_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, treatment_right, y_right, depth + 1)
        return node
    
    def _ATE(self, treatment, y):
        
        return (treatment * y).sum() / treatment.sum() - \
            ((1 - treatment) * y).sum() / (1 - treatment).sum()
        
    def _best_split(self, X, treatment, y):
        
        if (y.size < self.min_samples_leaf * 2) or \
            (np.sum(treatment) < self.min_samples_leaf_treated) or \
            (np.sum(1 - treatment) < self.min_samples_leaf_control):
            
            return None, None
        
        best_ddp = 0
        best_idx, best_thr = None, None
        
         # Loop through all features
        for idx in range(X.shape[1]):
            
            unique_values = np.unique(X[:, idx])
            if len(unique_values) > 10:
                percentiles = np.percentile(X[:, idx], [3, 5, 10, 20, 30, 50, 70, 80, 90, 95, 97])
            else:
                percentiles = np.percentile(unique_values, [10, 50, 90])
                
            threshold_options = np.unique(percentiles)
            
                      
            for thr_i in threshold_options:  # possible split positions
                
                indices_left = X[:, idx] <= thr_i
                treatment_left, y_left = \
                    treatment[indices_left], y[indices_left]
                treatment_right, y_right = \
                    treatment[~indices_left], y[~indices_left]
                
                ate_left = self._ATE(treatment_left, y_left)
                ate_right = self._ATE(treatment_right, y_right)
                
                ddp = np.abs(ate_right - ate_left)

                if (ddp > best_ddp) and \
                    (y_left.size >= self.min_samples_leaf) and \
                    (np.sum(treatment_left) >= self.min_samples_leaf_treated) and \
                    (np.sum(1 - treatment_left) >= self.min_samples_leaf_control) and \
                    (y_right.size >= self.min_samples_leaf) and \
                    (np.sum(treatment_right) >= self.min_samples_leaf_treated) and \
                    (np.sum(1 - treatment_right) >= self.min_samples_leaf_control):
                    
                    best_ddp = ddp
                    best_idx = idx
                    best_thr = thr_i

        return best_idx, best_thr
        
        
    
    def predict(self, X: np.ndarray) -> Iterable[float]:
        # compute predictions
        # pass
        return [self._predict(inputs) for inputs in X]

    def _predict(self, inputs):
        """Predict class for a single sample."""
        node = self.tree
        while node.left:
            if inputs[node.split_feat] < node.split_threshold:
#                 print("node.left ATE {}".format(node.ATE))
                node = node.left
            else:
#                 print("node.right ATE {}".format(node.ATE))
                node = node.right
        return node.ATE
    

class Node:
    def __init__(self, n_items, ATE):
        
        self.n_items = n_items
        self.ATE = ATE
        self.split_feat = None
        self.split_threshold = None
        self.left = None
        self.right = None
        

In [44]:
tst_tree0 = UpliftTreeRegressor()

In [45]:
%%time
tst_tree0.fit(X=ex_X, treatment=ex_treatment, y=ex_y)

CPU times: user 319 ms, sys: 24.3 ms, total: 343 ms
Wall time: 341 ms


In [46]:
ex_X[:10, :]

array([[ 0.40072881,  0.8084128 ,  0.94338544,  0.19877038, -2.89635463],
       [ 1.11588933, -2.1070723 , -0.5123218 , -0.20680012,  0.70560314],
       [-0.15370419, -0.01068205, -0.38601986,  0.27470753, -0.27732682],
       [-1.59841147,  0.08452897, -1.27173378, -1.55240958,  0.81976223],
       [ 2.3284045 , -1.73166287,  0.06939483,  0.33940273,  0.62947765],
       [-0.67759225,  0.88594583,  1.15538455, -0.90513451,  2.46840948],
       [-0.75023587,  0.69239455, -0.87968088,  1.76834074,  0.9134523 ],
       [ 0.12427588, -0.66091977,  0.28536053,  0.6166741 ,  0.12586261],
       [-0.27575139,  0.53094036,  1.10188594, -0.2348322 , -0.42786006],
       [ 0.97827404, -0.28237968,  0.06142647,  0.18545884,  1.03901775]])

In [47]:
%%time
tst_pred0 = tst_tree0.predict(X=ex_X)

CPU times: user 91.9 ms, sys: 0 ns, total: 91.9 ms
Wall time: 87.7 ms


In [48]:
tst_pred0

[0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 3.0993996235583836,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 2.499740934854509,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 -1.474148638300432,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.7590322044259551,
 0.75903220442

## draft

In [ ]:
### General fit scheme
## 1 
## 2
## 
##
##
##
##


In [28]:
from pandas import util

In [ ]:
# # Loop through all features.
# for idx in range(self.n_features_):
#     # Sort data along selected feature.
#     thresholds, classes = zip(*sorted(zip(X[:, idx], y)))

In [42]:
tst_df0 = util.testing.makeDataFrame()
# tst_df0.reset_index(inplace=True)
tst_df0.head()

,A,B,C,D
zAEskJfmce,0.350686,-0.840366,0.112155,0.089373
Tc8AKGraXO,0.221315,1.640683,-0.992910,-0.094026
VYcqvT0Vf8,-0.815825,-0.458344,0.541230,-0.053656
cbGGuh9z6b,-0.530466,1.135224,-0.723001,0.864190
Ziuy9vPJ9U,-0.940329,-1.948555,1.476857,1.536006


In [50]:
tst_df0.iloc[:, 2].head()

zAEskJfmce    0.112155
Tc8AKGraXO   -0.992910
VYcqvT0Vf8    0.541230
cbGGuh9z6b   -0.723001
Ziuy9vPJ9U    1.476857
Name: C, dtype: float64

In [64]:
# Loop through all features.
for idx in range(tst_df0.shape[1]):
    # Sort data along selected feature.
    thresholds, classes = zip(*sorted(zip(tst_df0.iloc[:, idx], tst_df0.iloc[:, 2])))
    print(thresholds[:5], classes[:5])

(-1.5301289637700553, -0.940329368954793, -0.815824868354525, -0.7718355581791742, -0.6297251037375826) (-0.0550790902641854, 1.476856936610013, 0.5412296425068981, -0.5813754264695793, 1.0282777893223407)
(-1.9485553172547707, -1.7453132664355626, -1.5571073528669792, -1.4113737294648265, -1.272299271398875) (1.476856936610013, -0.0550790902641854, 0.2123524186665515, 0.04377451425037322, -0.6280248927710689)
(-2.379753375527243, -1.8911033671672377, -1.7677990153239014, -1.625136472873728, -1.4042169066442236) (-2.379753375527243, -1.8911033671672377, -1.7677990153239014, -1.625136472873728, -1.4042169066442236)
(-1.7678346496722328, -1.7300898824934634, -1.235908229727449, -1.2162158591773953, -1.153289562186384) (-0.16203502689939805, 0.037867661018628276, -0.0550790902641854, -1.625136472873728, 1.5848901896037235)


In [84]:
# list(zip(*sorted(zip(tst_df0.iloc[:, idx], tst_df0.iloc[:, 2]))))[:5]

s1 = [4, 2, 3, 1]
s2 = ['f', 'b', 'a', 'c']
list(zip(s1, s2))

[(4, 'f'), (2, 'b'), (3, 'a'), (1, 'c')]

In [89]:
list(sorted(zip(s1, s2)))

[(1, 'c'), (2, 'b'), (3, 'a'), (4, 'f')]

In [91]:
list(zip(sorted(zip(s1, s2))))

[((1, 'c'),), ((2, 'b'),), ((3, 'a'),), ((4, 'f'),)]

In [92]:
list(zip(*sorted(zip(s1, s2))))

[(1, 2, 3, 4), ('c', 'b', 'a', 'f')]

In [94]:
print(sorted(zip(s1, s2)))
print(*sorted(zip(s1, s2)))

[(1, 'c'), (2, 'b'), (3, 'a'), (4, 'f')]
(1, 'c') (2, 'b') (3, 'a') (4, 'f')


In [131]:
print(s1)
print(s1[2])
print(s1[~2])

[4, 2, 3, 1]
3
2


In [134]:
ind_left = np.array(s1) < 3
ind_left

array([False,  True, False,  True])

In [137]:
np.array(s1)[ind_left]

array([2, 1])

In [138]:
np.array(s1)[~ind_left]

array([4, 3])

In [115]:
tst_treat = np.random.randint(0, 2, 10)
tst_y = np.random.randint(1, 5, 10)
print(tst_treat)
print(tst_y)

[1 1 1 0 0 0 0 0 1 1]
[3 4 4 1 3 4 3 1 2 2]


In [124]:
## ATE
(tst_treat * tst_y).sum() / tst_treat.sum() - \
    ((1 - tst_treat) * tst_y).sum() / (1 - tst_treat).sum()

0.6000000000000001